In [31]:
# Gradient Descent and Linear Regression with PyTorch

In [32]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy torch torchvision torchaudio

## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

SEE THIS FORMULAE YE EK MATRIX FORM KRRE HAI , YE 2 ROWS HAI ,AUR 3 COLUMNS HAI 
AGAR B1 B2 KO ALAG SE DEKHE TO YE EK VECTOR FORM KRTA HUA DIKHAYE DEGA 
```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The _learned_ weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region. 

We'll _train_ our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. Let's begin by importing Numpy and PyTorch.

In [33]:
import numpy as np
import torch

## Training data

We can represent the training data using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [34]:
# Model means (function or program)

In [35]:
# Jab [temperature,rainfaill,humidity] change hota hai ,tab apple or oranges k yield bhe change hoti hai
# Is relationship ko hum model me implement krna chahte hai
# Agar is relationship ko hum capture krle kisi model k andar ,tab hum us model ko use krsakte hai new regions
    #-me prediction krne k liye .

# Application of this model : iske hisab se hum capacity planning kr sakte hai ,
    #-kaha kya kab lagana chahie uske hisab se hum optimizations kr sakte hai 
    #-efficient business decisions le sakte hai 

# Linear regression me hum ye man k chalte hai ki ye sab factors aur inke results a linear combination hain .

# Isme hum ye mante hai  ki har ek jo target variable hai ,wo weighted sum hai (temp,rainfall,humidity ka)


# kisi situation me agar temperature ,humidity ,rainfall sab zero bhe hojaye to bhe crop zero nhi hota
    #-isliye hum ek constant rakhte use krte hai 

# apple k liye alag constants honge ,orange k liye alag constants honge 


# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [36]:
# THis means jo yield of apples hai ,wo ek linear aur plainer function temperature ,rainfall and humidity ka
# [see image above]so agar hum yield of apples ko plot krenge sirf (temperature and rainfall ) k hisab se
    #-coz agar hum humidity dikha denge to hume 4th dimension ki zarurat padegi ,yield dikhane k liye .
    #- abhi k liye hum humidity ko ignore kr sakte hai 

# Rainfall hogi to crop aache hoge ,zada rainfall pr crop kharab bhe hojayegi 
    #-Isliye ye sirf ek model hai , ye koi actual law nhi hai .
    #- isse certain range of values me use kr sakte hain 

In [37]:
# Learning part in linear regression :

# Learning part me hame figureout krna hai ek aacha set of weights
  
# yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
    #- iss equation me humne likh to diya hai but hame pata nhi hai ki aache weights kya hai 
    #- hamme janna hai ki aache set of weights kya honge 

# Hum training data k inputs and outputs ko use krke aache weights nikalenge 
    #-aur un weights ko use krke hum new data pr predictions krenge(jaha ki hame sirf temperature ,rainfall,humidity)
    #- pata hai. 

# ye hum kaise krenge , iske liye hum pehle kuch random weight se start krenge ,
    #- phir hum thoda thoda weights change krenge thoda thoda aache predictions krne k liye 
    #- step by step hum better predictions krte jayenge 

# hum ek algorithm/technique apply krenge weights ko improve krne k liye called(gradient descent )
    #- aur wo krte krte hamare weight improve hote chale jayenge 

# hum apne training data ko 2 matrices ki tarah represent kr sakte hai , 
    #- ek the hamare input variables  ek column hai ,5 regions k data point k liye 5 rows hai 


In [38]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

# NOTE : Hame mostly floating point number use krna chahie , 
    #-ek tarika hai ki matrix me .dal do 
    #- 2nd tareka ,specify explicitly numpy array me ,dtype='float32' ,ki aapko kis datatype ka data chahie 

# Targets means hamare input data k outputs     

We've separated the input and target variables because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors.

Let's convert the arrays to PyTorch tensors.

In [39]:
# Convert inputs and targets to tensors ie [convert numpy array to pytorch tensors]
inputs = torch.from_numpy(inputs) 
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [40]:
# AB hame model banana hai , hame model k liye 2 chize chahie ,weights and biases 

# Weights and biases


w = torch.randn(2, 3, requires_grad=True) # ye matrix create hota hai random number ka (2,3) ye shape hai 
# y ka derivative w.grad me store ho jayega 
b = torch.randn(2, requires_grad=True)
print(w) # weights
print(b) # biases 
# inhe random gausian destribution me se pick kiya jata hai (bell curve )(in -1 to 1 range )

tensor([[ 0.9141, -0.1542, -0.8167],
        [ 0.1032, -0.6356,  2.0342]], requires_grad=True)
tensor([-2.2624, -0.1949], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [41]:
w

tensor([[ 0.9141, -0.1542, -0.8167],
        [ 0.1032, -0.6356,  2.0342]], requires_grad=True)

In [42]:
inputs


tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [43]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [44]:
w.t()

tensor([[ 0.9141,  0.1032],
        [-0.1542, -0.6356],
        [-0.8167,  2.0342]], grad_fn=<TBackward0>)

In [45]:
# hamara model sirf ek function hai ,jo ki ek matrix multiplication perform krega 
def model(x):
    return x @ w.t() + b
# @ operator means matrix multiplication 
# w.t() , t method se matrix transpose hojate hai

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [46]:
# Generate predictions
preds = model(inputs)
print(preds) # ye outputs hamare model k predictions hai 

# model ne un random weights ko use krke predict kiya hai yeild of oranges and apples



tensor([[ 19.0162,  52.2222],
        [ 15.0808,  83.4502],
        [  9.2334,  41.5947],
        [ 54.1240,  58.2629],
        [-11.1620,  88.3013]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [47]:
# Compare with targets
print(targets)

# ye to random weights and biases k sath aaya that ,ab hum model ko dhere dhere improve krenge 

# isse pehle ki hum model ko improve kr paye hame batana padega model ko ki ye kitna kharab hai 
    #- jab pata hoga ki kitna kharab hai tabhi bata payenge kya change krna chahie .

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [92]:
diff = preds -targets # below put into a function
torch.sum(diff*diff)

tensor(583.7306, grad_fn=<SumBackward0>)

In [94]:
diff.numel() # numel function gives number of elements 

30

In [48]:
# kitna kharab hai target value predict krne k liye: 
    #- Calculate krne k liye hum difference lenge (element wise dono matrix ka )
    #- phir hum sare elements ko square kr denge , sare differences positive hojayenge ,aur uska average le lenge
    #- ab wo single number aagaya ,aur us single number ko use krsakte hai as an evaluation matrix 
    #- a.k.a Mean Squared Error 

# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [95]:
# Compute loss
loss = mse(preds, targets)
print(loss)

# How to interpret this result
    #-  if we take square root of this 19.4577 =4.45, so that means each prediction(of each element ) ,
    #- differ from  the output by (4.45 ) on an average .


# In this case 3,4,5 is a decent error     

tensor(19.4577, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [50]:
# Compute gradients
loss.backward() 

# pytorch se ye sare calculations track krke rakhe the hamare liye 
# jab loss.backward call hota hai , tab pytorch dekhta hai wo kon kon se tensors hai ,jisme requires grad true
    #-set ki hue the,wo sare derivates save ho chuke hai w.grad me .

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [51]:
# Gradients for weights

# # Ye hamara original weight hai 
# tensor([[ 0.9141, -0.1542, -0.8167],
#         [ 0.1032, -0.6356,  2.0342]], requires_grad=True)


# derivative of first loss with respect to this weight is this 
# tensor([[-4569.7358, -6513.1475, -3795.6680],
#         [-2159.2917, -3403.3125, -1710.2628]])


print(w)
print(w.grad)

# do this with pen and paper 

tensor([[ 0.9141, -0.1542, -0.8167],
        [ 0.1032, -0.6356,  2.0342]], requires_grad=True)
tensor([[-4569.7358, -6513.1475, -3795.6680],
        [-2159.2917, -3403.3125, -1710.2628]])


In [97]:
print(inputs)
print(inputs.grad) # ye none aayega  ,coz hamne requires grad true set nhi kiya tha 

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])
None


In [96]:
print(b)
print(b.grad)

tensor([-2.2628, -0.1967], requires_grad=True)
tensor([0., 0.])


In [ ]:
# Note
# Quadratic function me waise to sirf ek minima hota hai 
# polynomial function me many minima hote hai 

# see below diagram : kisi ek weight k liye w11 loss ye hai (on graph )
    #-see in graph w11 ko change krke loss kaise change hora ,
    #- NOTE : koi ek range hai jaha pr w11 ki value sabse low hai <usko kaise identify karenge>

# <usko identify krne k liye>
# w11 pr aao , w11 pr jo current value hai loss ki , aur jo current gradient hai loss ka w11 k sath 
    #-dy/dy ek point pr curve ka slope batata hai 

    #- if gradient is positive -> slope is increasing
    #- if gradient is negative  -> slope is decreasing <iss fact ko hum use karenge apne weight ko imorove krenge>






## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [52]:
w
w.grad

tensor([[-4569.7358, -6513.1475, -3795.6680],
        [-2159.2917, -3403.3125, -1710.2628]])

In [53]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [54]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(4220.0220, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [55]:
w.grad.zero_() # ek round of calculation hogaya hai ,next round k liye values ko zero se reinitialize krre hai
b.grad.zero_()
print(w.grad)
print(b.grad)

# kabhi bhie gradient computation krke kam hogaye , to zero se initialze krdo 

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [56]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[28.3486, 56.8144],
        [27.4007, 89.5050],
        [24.1388, 49.0260],
        [62.9907, 62.5618],
        [ 0.9013, 94.2559]], grad_fn=<AddBackward0>)


In [57]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(3336.3821, grad_fn=<DivBackward0>)


In [58]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-3602.5527, -5463.9307, -3150.3130],
        [-1682.8739, -2884.7578, -1391.8906]])
tensor([-47.4440, -21.5674])


Let's update the weights and biases using the gradients computed above.

In [59]:
# HAME GRADIENT K OPPOSITE DIRECTION ME EK CHOOT SA STEP LENA HAI , 

# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5 # har ek element contribute kre thoda thoda loss ko kam krne me 
    b -= b.grad * 1e-5 # bias and weigts update hogaye hain
    w.grad.zero_() # ek bar weight and bias update hogaye tab hamne unke gradients ko hata diya(zero kr diya)
    b.grad.zero_()

Let's take a look at the new weights and biases.

In [60]:
print(w)
print(b)

tensor([[ 0.9958, -0.0344, -0.7473],
        [ 0.1416, -0.5727,  2.0652]], requires_grad=True)
tensor([-2.2614, -0.1944], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [61]:
# Calculate loss
preds = model(inputs) # weights and biases update hone k bad new predictions .
loss = mse(preds, targets)
print(loss)

tensor(2735.3074, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [62]:
# ek bar pure process ko repeat krne ko ek epochs kehte hai 
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

# Gradient descent : hamne weights ko thoda thoda kam kiya taki loss thoda thoda kam hote jaye
    #- at the end hamne weights ko bahut aacha krdiya           

    #- hamne bina actual relationship ko jane hue ,hamne us relationship ko ,improve krdiya hai 
    # - hamne sirf ek model banaya tha us relationship ka      

Once again, let's verify that the loss is now lower:

In [63]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(526.1099, grad_fn=<DivBackward0>)


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [64]:
# Predictions
preds

tensor([[ 64.9523,  73.9559],
        [ 79.9299, 113.3156],
        [111.3457,  98.3353],
        [ 67.7820,  59.4283],
        [ 70.2602, 128.0186]], grad_fn=<AddBackward0>)

In [65]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs. 

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [ ]:
# In above all ,hamne implement kiya linear regression ko basic tensor operation use krke 

# Lakin kyuki ye bahut common pattern hai : pytorch me ye buildin functionality hai(buildin functions & classes hai)
    #- taki aap ye aasani se kr paye (model trainig and model definition )



In [98]:
# humn torch.nn to import krke linear regression to implement krenge chote se code me 

import torch.nn as nn  # nn commonly used alias

As before, we represent the inputs and targets and matrices.

In [99]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [71]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [100]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [ ]:
# NOTE : Real life me hame 10 lakh ,2 lakh number of data points milte hai,
    #-  itne sare ko hum ek sath process nhi kr sakte hai
    #- isliye hum data ko chote chote batches me splite kr lete hain 

# so hum 15 rows ko 5 rows k batches me process krenge 

# batches me process krne se computation fast hoti hai , kyuki kam calculation krne hoti hai ,faster training
# shayad ram me fit bhe na hoga aapka pura dataset

# batches me krke k liye : hame dataset and dataloader banana hoga 

We are using 15 training examples to illustrate how to work with large datasets in small batches. 

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [72]:
from torch.utils.data import TensorDataset
#        MODULE                 CLASS

In [73]:
# Define dataset
train_ds = TensorDataset(inputs, targets) # tensor dataset object 
# tensor dataset object : ek standard api deta hai
# agar aap inputs and targets pass krte hai ,ek input dataset ko to ye ek tensor dataset object ban jata hai
  
# tensor dataset object : ek aasan tareka deta hai ,ek standard api deta hai .agar aapko apne pure data me se
                                                                        #- kuch slice of data nikalna hai to      

train_ds[0:3]  #(2 datasets aajate hai , input ,outputs)

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [74]:
from torch.utils.data import DataLoader
# data loader hamare data ko batches me split krta hain 
# agar hamare pass 15 elements hai to hum 15 elements ko 3 batches me split krenge 

In [75]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [76]:
# dataloaders : shuffle krke data ko batches me banadete hain  , (shuffle se loss reduction faster ho jata hai)
# dataloaders ko hum generally hum ek for loop me use krte hain ,

# xb -> batch of inputs 
# yb -> batch of outputs/targets
for xb, yb in train_dl:
    print(xb) 
    print(yb)
    # corresponding iputs k targets print hue hain
    break

tensor([[ 88., 134.,  59.],
        [ 91.,  87.,  65.],
        [103.,  43.,  36.],
        [ 92.,  87.,  64.],
        [ 87., 134.,  58.]])
tensor([[118., 132.],
        [ 80., 102.],
        [ 20.,  38.],
        [ 82., 100.],
        [119., 133.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [77]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5175,  0.3125, -0.2209],
        [ 0.0248,  0.2029,  0.2904]], requires_grad=True)
Parameter containing:
tensor([-0.1309,  0.3010], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [78]:
# Parameters
list(model.parameters()) # parameters gives model k andar kya kya hain

[Parameter containing:
 tensor([[-0.5175,  0.3125, -0.2209],
         [ 0.0248,  0.2029,  0.2904]], requires_grad=True),
 Parameter containing:
 tensor([-0.1309,  0.3010], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [79]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-26.4744,  28.1932],
        [-33.8673,  38.9993],
        [-16.0968,  46.4897],
        [-47.6576,  22.3007],
        [-21.3069,  41.8194],
        [-27.3045,  28.0151],
        [-34.4007,  39.0869],
        [-16.8352,  46.8050],
        [-46.8276,  22.4788],
        [-21.0102,  42.0850],
        [-27.0078,  28.2808],
        [-34.6973,  38.8212],
        [-15.5633,  46.4022],
        [-47.9542,  22.0351],
        [-20.4768,  41.9974]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [80]:
# Import nn.functional
import torch.nn.functional as F # F alias convention hai

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [81]:
# Define loss function
loss_fn = F.mse_loss # mse_loss is loss function hai ,jo already implemented hai 

Let's compute the loss for the current predictions of our model.

In [82]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(7761.6987, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [83]:
# Define optimizer : for gradient descent 

#  optim : class me bahut sare optimizers hai 
# SGD : ek type ka optimizer hai jiske full form hai stochastic gradient descent 
        #- stochastic means  data ko random batches me select kiya gaya hai 

# ab is optimizer ko 2 chize pass kene hai ,
    #- pehle chiz (kon kon se weights , biases matrices )per optimization perform krne hai   
    #- dusre chiz learning rate : lr ,ye number multiply hoga gradient k sath 
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [102]:
# STEP BY STEP GRADIENT DESCENT APPLY HOTA RAHEGA AUR MODEL IMPROVE HOTA RAHEGA

# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()



        
        # Print the progress
        if (epoch+1) % 10 == 0: # HAR BAR CURRENT EPOCHS K LOSS KO PRINT KR DENGE 
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [85]:
fit(100, model, loss_fn, opt, train_dl) 

# YE LOSS MEAN SQUARED ERROR HAIN 

Epoch [10/100], Loss: 249.7894
Epoch [20/100], Loss: 17.7657
Epoch [30/100], Loss: 34.7012
Epoch [40/100], Loss: 58.9675
Epoch [50/100], Loss: 52.1314
Epoch [60/100], Loss: 41.1994
Epoch [70/100], Loss: 24.5418
Epoch [80/100], Loss: 21.6236
Epoch [90/100], Loss: 18.7916
Epoch [100/100], Loss: 23.7108


Let's generate predictions using our model and verify that they're close to our targets.

In [86]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.6564,  71.4103],
        [ 79.1703,  99.0571],
        [123.5475, 134.4507],
        [ 24.7215,  43.2086],
        [ 94.5800, 112.8063],
        [ 56.4096,  70.4205],
        [ 78.4705,  98.8337],
        [123.5560, 134.9093],
        [ 25.9682,  44.1985],
        [ 95.1269, 113.5727],
        [ 56.9565,  71.1869],
        [ 77.9235,  98.0672],
        [124.2473, 134.6741],
        [ 24.1745,  42.4422],
        [ 95.8268, 113.7961]], grad_fn=<AddmmBackward0>)

In [87]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [88]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.4855, 68.3712]], grad_fn=<AddmmBackward0>)

The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)

Keep this picture in mind as you work through the next few tutorials. 



## Exercises and Further Reading

We've covered the following topics in this tutorial:

- Introduction to linear regression and gradient descent
- Implementing a linear regression model using PyTorch tensors
- Training a linear regression model using the gradient descent algorithm
- Implementing gradient descent and linear regression using PyTorch built-in


Here are some resources for learning more about linear regression and gradient descent:

* An visual & animated explanation of gradient descent: https://www.youtube.com/watch?v=IHZwWFHWa-w

* For a more detailed explanation of derivates and gradient descent, see [these notes from a Udacity course](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf). 

* For an animated visualization of how linear regression works, [see this post](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://github.com/Cleo-Stanford-CS/CS229_Notes/blob/master/lectures/cs229-notes1.pdf) from CS229 at Stanford University.

* To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://www.kaggle.com/c/boston-housing) competition on Kaggle, a website that hosts data science competitions.

With this, we complete our discussion of linear regression in PyTorch, and we’re ready to move on to the next topic: [Working with Images & Logistic Regression](https://jovian.ai/aakashns/03-logistic-regression).

## Questions for Review

Try answering the following questions to test your understanding of the topics covered in this notebook:

1. What is a linear regression model? Give an example of a problem formulated as a linear regression model.
2. What are input and target variables in a dataset? Give an example.
3. What are weights and biases in a linear regression model?
4. How do you represent tabular data using PyTorch tensors?
5. Why do we create separate matrices for inputs and targets while training a linear regression model?
6. How do you determine the shape of the weights matrix & bias vector given some training data?
7. How do you create randomly initialized weights & biases with a given shape?
8. How is a linear regression model implemented using matrix operations? Explain with an example.
9. How do you generate predictions using a linear regression model?
10. Why are the predictions of a randomly initialized model different from the actual targets?
11. What is a loss function? What does the term “loss” signify?
12. What is mean squared error?
13. Write a function to calculate mean squared using model predictions and actual targets.
14. What happens when you invoke the `.backward` function on the result of the mean squared error loss function?
15. Why is the derivative of the loss w.r.t. the weights matrix itself a matrix? What do its elements represent?
16. How is the derivate of the loss w.r.t. a weight element useful for reducing the loss? Explain with an example.
17. Suppose the derivative  of the loss w.r.t. a weight element is positive. Should you increase or decrease the element’s value slightly to get a lower loss?
18. Suppose the derivative  of the loss w.r.t. a weight element is negative. Should you increase or decrease the element’s value slightly to get a lower loss?
19. How do you update the weights and biases of a model using their respective gradients to reduce the loss slightly?
20. What is the gradient descent optimization algorithm? Why is it called “gradient descent”?
21. Why do you subtract a “small quantity” proportional to the gradient from the weights & biases, not the actual gradient itself?
22. What is learning rate? Why is it important?
23. What is `torch.no_grad`?
24. Why do you reset gradients to zero after updating weights and biases?
25. What are the steps involved in training a linear regression model using gradient descent?
26. What is an epoch?
27. What is the benefit of training a model for multiple epochs?
28. How do you make predictions using a trained model?
29. What should you do if your model’s loss doesn’t decrease while training? Hint: learning rate.
30. What is `torch.nn`?
31. What is the purpose of the `TensorDataset` class in PyTorch? Give an example.
32. What is a data loader in PyTorch? Give an example.
33. How do you use a data loader to retrieve batches of data?
34. What are the benefits of shuffling the training data before creating batches?
35. What is the benefit of training in small batches instead of training with the entire dataset?
36. What is the purpose of the `nn.Linear` class in PyTorch? Give an example.
37. How do you see the weights and biases of a `nn.Linear` model?
38. What is the purpose of the `torch.nn.functional` module?
39. How do you compute mean squared error loss using a PyTorch built-in function?
40. What is an optimizer in PyTorch?
41. What is `torch.optim.SGD`? What does SGD stand for?
42. What are the inputs to a PyTorch optimizer? 
43. Give an example of creating an optimizer for training a linear regression model.
44. Write a function to train a `nn.Linear` model in batches using gradient descent.
45. How do you use a linear regression model to make predictions on previously unseen data?

